In [1]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv(
    "../data/processed/inkar_nord_bayern_arbeitsmarkt_2022.csv",
    sep=";"
)

# 1. NaN-Zeile entfernen
df = df.dropna(subset=["Raumeinheit"])

# 2. Spaltennamen säubern (gegen Leerzeichen/Sonderzeichen)
df.columns = df.columns.str.strip()

# Spaltennamen säubern
df.columns = df.columns.astype(str).str.strip()

# Einheitlich umbenennen
df = df.rename(columns={
    "Beschäftigtenquote Frauen": "BQ_Frauen",
    "Beschäftigtenquote Männer": "BQ_Männer",
    "Teilzeitbeschäftige Frauen": "TZ_Frauen",
    "Teilzeitbeschäftigte": "TZ_Gesamt"
})

df

,Kennziffer,Raumeinheit,Aggregat,TZ_Frauen,TZ_Gesamt,BQ_Männer,BQ_Frauen
1,93.0,Oberpfalz,Regierungsbezirke,"52,69","29,27","71,71","63,67"
2,94.0,Oberfranken,Regierungsbezirke,"52,89","30,48","70,95","65,11"
3,95.0,Mittelfranken,Regierungsbezirke,"50,09","29,68","72,06","64,12"
4,96.0,Unterfranken,Regierungsbezirke,"53,99","30,34","72,6","63,82"


In [2]:
for col in ["BQ_Frauen","BQ_Männer","TZ_Frauen","TZ_Gesamt"]:
    df[col] = df[col].astype(str).str.replace(",", ".", regex=False).astype(float)

df.dtypes

Kennziffer     float64
Raumeinheit     object
Aggregat        object
TZ_Frauen      float64
TZ_Gesamt      float64
BQ_Männer      float64
BQ_Frauen      float64
dtype: object

In [3]:
import plotly.express as px

df_long_tz = df.melt(
    id_vars="Raumeinheit",
    value_vars=["TZ_Frauen", "TZ_Gesamt"],
    var_name="Kategorie",
    value_name="Teilzeitquote"
)

df_long_tz["Kategorie"] = df_long_tz["Kategorie"].replace({
    "TZ_Frauen": "Frauen",
    "TZ_Gesamt": "Gesamt"
})

df_long_tz


,Raumeinheit,Kategorie,Teilzeitquote
0,Oberpfalz,Frauen,52.69
1,Oberfranken,Frauen,52.89
2,Mittelfranken,Frauen,50.09
3,Unterfranken,Frauen,53.99
4,Oberpfalz,Gesamt,29.27
5,Oberfranken,Gesamt,30.48
6,Mittelfranken,Gesamt,29.68
7,Unterfranken,Gesamt,30.34


In [4]:
fig = px.bar(
    df_long_tz,
    x="Raumeinheit",
    y="Teilzeitquote",
    color="Kategorie",
    barmode="group",
    height=400,
    title="Teilzeitbeschäftigung: Frauen vs. Gesamt (Nordbayern, 2022)"
)

fig.update_traces(
    texttemplate="%{y:.2f}%",
    textposition="outside"
)

fig.update_yaxes(range=[0, 60])

fig.update_layout(
    xaxis_title="Regierungsbezirk",
    yaxis_title="Teilzeitquote in %",
)

fig.update_traces(textfont_size=12)
fig.update_layout(title_x=0.5)

# Quelle 
fig.add_annotation(
    x=1.02,
    y=-0.30,
    xref="paper",
    yref="paper",
    text="Quelle: Inkar",
    showarrow=False,
    font=dict(size=12, color="gray"),
    xanchor="left"
)

fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    # Spaltennamen vereinheitlichen (ä->ae etc.) 
    df = df.copy()
    df.columns = (
        df.columns.astype(str)
        .str.strip()
        .str.replace("Ä", "Ae").str.replace("Ö", "Oe").str.replace("Ü", "Ue")
        .str.replace("ä", "ae").str.replace("ö", "oe").str.replace("ü", "ue")
        .str.replace("ß", "ss")
    )
    return df

def load_inkar(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, sep=";", decimal=",", encoding="utf-8-sig")
    df = normalize_columns(df)

    # 1) Meta-Zeilen entfernen
    df = df.dropna(subset=["Raumeinheit"])

    return df

def to_numeric_cols(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    df = df.copy()
    for c in cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

# Dateien laden 
df_nord = load_inkar("../data/processed/inkar_nord_bayern_arbeitsmarkt_2022.csv")
df_sued = load_inkar("../data/processed/inkar_südbayern_arbeitsmarkt_2022.csv")

# Zielspalten (Nordbayern nutzt ae)
col_frauen = "Teilzeitbeschaeftige Frauen"
col_gesamt = "Teilzeitbeschaeftigte"

# sicherstellen, dass es Zahlen sind
df_nord = to_numeric_cols(df_nord, [col_frauen, col_gesamt])
df_sued = to_numeric_cols(df_sued, [col_frauen, col_gesamt])

# Mittelwerte je Region
north_vals = df_nord[[col_frauen, col_gesamt]].mean(numeric_only=True)
south_vals = df_sued[[col_frauen, col_gesamt]].mean(numeric_only=True)

print("Nordbayern means:\n", north_vals)
print("Südbayern means:\n", south_vals)

# Vergleichstabelle
df_compare = pd.DataFrame({
    "Region": ["Nordbayern", "Südbayern"],
    "Frauen": [north_vals[col_frauen], south_vals[col_frauen]],
    "Gesamt": [north_vals[col_gesamt], south_vals[col_gesamt]],
})

display(df_compare)

# Plot 
fig = go.Figure()

fig.add_bar(
    x=df_compare["Region"], y=df_compare["Frauen"],
    name="Frauen", marker_color="#1f77b4",
    text=df_compare["Frauen"].round(2), textposition="outside"
)

fig.add_bar(
    x=df_compare["Region"], y=df_compare["Gesamt"],
    name="Gesamt", marker_color="#d62728",
    text=df_compare["Gesamt"].round(2), textposition="outside"
)

fig.update_layout(
    title={"text": "Teilzeitbeschäftigung: Frauen vs. Gesamt (Nordbayern vs. Südbayern, 2022)", "x": 0.5},
    xaxis_title="Region",
    yaxis_title="Teilzeitquote in %",
    barmode="group",
    template="plotly_white",
    height=520,
    legend_title="Kategorie",
)

fig.update_yaxes(range=[0, max(df_compare["Frauen"].max(), df_compare["Gesamt"].max()) * 1.2])

# Quelle 
fig.add_annotation(
    x=1.03,
    y=-0.20,
    xref="paper",
    yref="paper",
    text="Quelle: Inkar",
    showarrow=False,
    font=dict(size=12, color="gray"),
    xanchor="left"
)
fig.show()


Nordbayern means:
 Teilzeitbeschaeftige Frauen    52.4150
Teilzeitbeschaeftigte          29.9425
dtype: float64
Südbayern means:
 Teilzeitbeschaeftige Frauen    50.143333
Teilzeitbeschaeftigte          28.370000
dtype: float64


,Region,Frauen,Gesamt
0,Nordbayern,52.415000,29.9425
1,Südbayern,50.143333,28.3700


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Data: Germany 2022
df_2022 = pd.DataFrame({
    "Geschlecht": ["Frauen", "Männer"],
    "Teilzeitquote (%)": [47.4, 10.9]
})

fig = go.Figure()

fig.add_bar(
    x=df_2022["Geschlecht"],
    y=df_2022["Teilzeitquote (%)"],
    text=[f"{v:.1f}%" for v in df_2022["Teilzeitquote (%)"]],
    textposition="outside",
    marker_color=["#1f77b4", "#d62728"],  # blue = women, red = men
)

fig.update_traces(
    texttemplate="%{y:.1f}%",
    textposition="outside",
    textfont=dict(
        size=18,          
        color="black"     
    )
)

fig.update_xaxes(
    tickfont=dict(
        size=16   
    )
)

fig.update_yaxes(range=[0, 55], ticksuffix="%")

fig.update_layout(
    title={
        "text": "Teilzeitbeschäftigungsquote nach Geschlecht in Deutschland (2022)",
        "x": 0.5,
        "xanchor": "center"
    },
    xaxis_title="Geschlecht",
    yaxis_title="Teilzeitquote (%)",
    template="plotly_white",
    height=450,
    margin=dict(l=60, r=40, t=80, b=80),  
    annotations=[
        dict(
            text="Quelle: Destatis / Statista (2022)",
            x=0,
            y=-0.18,
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=11, color="gray"),
            align="left"
        )
    ]
)

fig.show()


In [ ]:
import plotly.graph_objects as go

# Daten (2022) – Frauen 18–64, mit/ohne Kinder
groups = [
    "Frauen ohne Kinder",
    "Frauen mit Kindern",
    "davon jüngstes Kind: unter 6 Jahre",
    "davon jüngstes Kind: 6 bis unter 18 Jahre",
]

paid_str = ["22:24", "17:42", "12:53", "21:29"]     # Erwerbsarbeit
unpaid_str = ["24:19", "39:23", "48:14", "32:25"]   # unbezahlte Arbeit
total_str = ["46:43", "57:05", "61:07", "53:55"]    # Summe

def hhmm_to_hours(hhmm: str) -> float:
    h, m = hhmm.split(":")
    return int(h) + int(m)/60

paid = [hhmm_to_hours(x) for x in paid_str]
unpaid = [hhmm_to_hours(x) for x in unpaid_str]
total = [p + u for p, u in zip(paid, unpaid)]

groups_r = groups[::-1]
paid_r = paid[::-1]
unpaid_r = unpaid[::-1]
paid_str_r = paid_str[::-1]
unpaid_str_r = unpaid_str[::-1]
total_str_r = total_str[::-1]
total_r = total[::-1]

# Plot
fig = go.Figure()

# Erwerbsarbeit (blau)
fig.add_trace(go.Bar(
    y=groups_r,
    x=paid_r,
    orientation="h",
    name="Erwerbsarbeit",
    marker_color="#0B4FA8",  
    text=paid_str_r,
    textposition="inside",
    insidetextanchor="middle",
    textfont=dict(color="white", size=14),
    hovertemplate="%{y}<br>Erwerbsarbeit: %{text}<extra></extra>",
))

# Unbezahlte Arbeit (rot)
fig.add_trace(go.Bar(
    y=groups_r,
    x=unpaid_r,
    orientation="h",
    name="Unbezahlte Arbeit",
    marker_color="#E85A70",  
    text=unpaid_str_r,
    textposition="inside",
    insidetextanchor="middle",
    textfont=dict(color="white", size=14),
    hovertemplate="%{y}<br>Unbezahlte Arbeit: %{text}<extra></extra>",
))

# Gesamtsumme rechts am Balkenende (Annotationen)
for y, t_hours, t_label in zip(groups_r, total_r, total_str_r):
    fig.add_annotation(
        x=t_hours + 0.25,  
        y=y,
        text=t_label,
        showarrow=False,
        font=dict(size=14, color="#1f1f1f"),
        xanchor="left",
        yanchor="middle"
    )

# Layout
fig.update_layout(
    barmode="stack",
    template="plotly_white",
    title=dict(
        text="Erwerbsarbeit und unbezahlte Arbeit von Frauen (2022)",
        x=0.5
    ),
    xaxis_title="Stunden pro Woche",
    yaxis_title="",
    height=650,
    legend=dict(
        title="",
        orientation="h",
        x=0.02,
        y=-0.14
    ),
    margin=dict(l=180, r=60, t=90, b=110),
)

# X-Achse sinnvoll begrenzen (bis knapp über Maximum)
fig.update_xaxes(range=[0, max(total) + 4], gridcolor="rgba(0,0,0,0.08)")

# Quelle
fig.add_annotation(
    x=0.60,
    y=-0.19,
    xref="paper",
    yref="paper",
    text="Quelle: Zeitverwendungserhebung 2022 (Destatis)",
    showarrow=False,
    font=dict(size=12, color="gray"),
    xanchor="left"
)

fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Helper: Stunden (dezimal) -> "hh:mm"
def hours_to_hhmm(x):
    total_minutes = int(round(float(x) * 60))
    h = total_minutes // 60
    m = total_minutes % 60
    return f"{h:02d}:{m:02d}"

# Daten 
data = {
    "Bereich": [
        "Zubereitung von Mahlzeiten / Küche",
        "Instandhaltung Haus/Wohnung, Textilien",
        "Einkaufen / Haushaltsorganisation",
        "Gartenarbeit / Bauen / Handwerk",
        "Betreuung / Pflege / Unterstützung",
        "Ehrenamt / Unterstützung anderer Haushalte",
        "Wegezeiten unbezahlte Arbeit"
    ],
    "Frauen": [6.74, 6.47, 4.90, 3.30, 3.65, 1.93, 2.88],
    "Männer": [3.65, 2.85, 4.00, 3.97, 1.92, 1.90, 2.42]
}
df = pd.DataFrame(data)

colors = [
    "#0B3C78",  # dunkelblau
    "#F45B69",  # rot
    "#3F73B5",  # blau
    "#E9808B",  # hellrot
    "#67A2E5",  # hellblau
    "#FBC4C4",  # rosa
    "#9DC9FE"   # sehr hellblau
]

fig = go.Figure()

# Balken:
for j in range(len(df)):
    fig.add_bar(
        y=["Männer"],
        x=[df.loc[j, "Männer"]],
        name=df.loc[j, "Bereich"],
        orientation="h",
        marker_color=colors[j],
        text=[hours_to_hhmm(df.loc[j, "Männer"])],
        textposition="inside",
        insidetextanchor="middle",
        textfont=dict(color="white"),
        showlegend=True
    )

for j in range(len(df)):
    fig.add_bar(
        y=["Frauen"],
        x=[df.loc[j, "Frauen"]],
        name=df.loc[j, "Bereich"],
        orientation="h",
        marker_color=colors[j],
        text=[hours_to_hhmm(df.loc[j, "Frauen"])],
        textposition="inside",
        insidetextanchor="middle",
        textfont=dict(color="white"),
        showlegend=False
    )

fig.update_layout(
    title=dict(
        text="Unbezahlte Arbeit 2022 – nach Arbeitsbereichen",
        x=0.5,
        xanchor="center"
    ),
    barmode="stack",
    template="plotly_white",
    height=520,
    margin=dict(l=80, r=40, t=80, b=70),
    xaxis_title="Stunden pro Woche",
    yaxis_title="",
    legend_title="Arbeitsbereich",
    
    # Quelle 
    annotations=[
        dict(
            text="Quelle: Zeitverwendungserhebung 2022 (Destatis)",
            x=1.45,
            y=-0.12,
            xref="paper",
            yref="paper",
            xanchor="right",
            showarrow=False,
            font=dict(size=11, color="gray"),
            align="right"
        )
    ]
)

fig.update_layout(legend_traceorder="reversed")

fig.show()